In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1
Default GPU Device: /gpu:0


In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    return input, keep_prob, layer3_out, layer4_out, layer7_out

print("VGG16 model is now loaded....")
tests.test_load_vgg(load_vgg, tf)

VGG16 model is now loaded....
Tests Passed


In [4]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    # Perform 1 x 1 conv for layer 7
    conv_1x1_layer_7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1,
                                   padding='same',
                                   kernel_initializer=tf.random_normal_initializer(stddev=0.01),
                                   kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    # Upsample the image
    output_7 = tf.layers.conv2d_transpose(conv_1x1_layer_7, num_classes, 4,
                                             strides=(2, 2),
                                             padding='same',
                                             kernel_initializer=tf.random_normal_initializer(stddev=0.01),
                                             kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))

    # Perform 1 x 1 conv for layer 4
    conv_1x1_layer_4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1,
                                   padding='same',
                                   kernel_initializer=tf.random_normal_initializer(stddev=0.01),
                                   kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    # Skip connection
    output_4_skip = tf.add(output_7, conv_1x1_layer_4)

    # Upsample the image layer 4
    output_4 = tf.layers.conv2d_transpose(output_4_skip, num_classes, 4,
                                          strides=(2, 2),
                                            padding='same',
                                            kernel_initializer=tf.random_normal_initializer(stddev=0.01),
                                            kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))

    # Perform 1 x 1 conv for layer 3
    conv_1x1_layer_3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1,
                                   padding='same',
                                   kernel_initializer=tf.random_normal_initializer(stddev=0.01),
                                   kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    # Skip connection
    output_3_skip = tf.add(output_4, conv_1x1_layer_3)

    # Upsample the image layer 4
    final_layer = tf.layers.conv2d_transpose(output_3_skip, num_classes, 16,
                                          strides=(8, 8),
                                          padding='same',
                                          kernel_initializer=tf.random_normal_initializer(stddev=0.01),
                                          kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    return final_layer
print("Layers defined...")
tests.test_layers(layers)

Layers defined...
Tests Passed


In [5]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    # Define and reshape logits
    logits = tf.reshape(nn_last_layer, (-1, num_classes))

    #Define and reshape labels
    labels = tf.reshape(correct_label, (-1, num_classes))

    # Loss function
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label))

    # Define training operation using adam optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(cross_entropy_loss)

    return logits, train_op, cross_entropy_loss
print("Optimization done...")
tests.test_optimize(optimize)

Optimization done...
Tests Passed


In [6]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    
    """
    sess.run(tf.global_variables_initializer())
    # TODO: Implement function
    for ii in range(epochs):
        print("Epoch {} ...".format(ii + 1))
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss],
                               feed_dict={input_image: image, correct_label: label, keep_prob: 0.5,
                                          learning_rate: 1e-4})
            print("Loss: = {:.3f}".format(loss))
        print()
       
    
#tests.test_train_nn(train_nn)


starting train_nn...
Done


In [7]:
def run():
    print("Let's begin...")
    num_classes = 2
    epochs = 25
    batch_size = 5
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)
    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network
        # TODO: Build NN using load_vgg, layers, and optimize function
        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes])
        learning_rate = tf.placeholder(tf.float32)

        input_image, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)

        layer_output = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)

        logits, train_op, cross_entropy_loss = optimize(layer_output, correct_label, learning_rate, num_classes)
        

        # TODO: Train NN using the train_nn function

        print('Start Training...')
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
                 correct_label, keep_prob, learning_rate)

        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        # OPTIONAL: Apply the trained model to a video





In [8]:
run()

Let's begin...
Tests Passed
Downloading...
Downlaoding done...

INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
Start Training...
Epoch 1 ...
Loss: = 1.468
Loss: = 1.293
Loss: = 1.074
Loss: = 1.025
Loss: = 0.942
Loss: = 0.850
Loss: = 0.809
Loss: = 0.815
Loss: = 0.780
Loss: = 0.767
Loss: = 0.740
Loss: = 0.734
Loss: = 0.737
Loss: = 0.724
Loss: = 0.710
Loss: = 0.711
Loss: = 0.708
Loss: = 0.703
Loss: = 0.704
Loss: = 0.695
Loss: = 0.694
Loss: = 0.692
Loss: = 0.686
Loss: = 0.681
Loss: = 0.680
Loss: = 0.674
Loss: = 0.669
Loss: = 0.657
Loss: = 0.660
Loss: = 0.659
Loss: = 0.640
Loss: = 0.635
Loss: = 0.626
Loss: = 0.636
Loss: = 0.615
Loss: = 0.609
Loss: = 0.600
Loss: = 0.570
Loss: = 0.555
Loss: = 0.540
Loss: = 0.551
Loss: = 0.534
Loss: = 0.541
Loss: = 0.504
Loss: = 0.455
Loss: = 0.452
Loss: = 0.419
Loss: = 0.436
Loss: = 0.375
Loss: = 0.382
Loss: = 0.403
Loss: = 0.416
Loss: = 0.395
Loss: = 0.363
Loss: = 0.387
Loss: = 0.316
Loss: = 0.331
Loss: = 0.319

Epoch 2 ...
Loss:

Loss: = 0.045
Loss: = 0.037
Loss: = 0.067
Loss: = 0.047
Loss: = 0.060
Loss: = 0.066
Loss: = 0.052
Loss: = 0.052
Loss: = 0.041
Loss: = 0.065
Loss: = 0.047
Loss: = 0.075
Loss: = 0.047
Loss: = 0.031
Loss: = 0.028

Epoch 11 ...
Loss: = 0.054
Loss: = 0.038
Loss: = 0.054
Loss: = 0.068
Loss: = 0.046
Loss: = 0.059
Loss: = 0.034
Loss: = 0.048
Loss: = 0.049
Loss: = 0.071
Loss: = 0.052
Loss: = 0.064
Loss: = 0.073
Loss: = 0.048
Loss: = 0.037
Loss: = 0.065
Loss: = 0.071
Loss: = 0.048
Loss: = 0.055
Loss: = 0.050
Loss: = 0.071
Loss: = 0.047
Loss: = 0.044
Loss: = 0.040
Loss: = 0.051
Loss: = 0.041
Loss: = 0.042
Loss: = 0.058
Loss: = 0.035
Loss: = 0.046
Loss: = 0.043
Loss: = 0.032
Loss: = 0.055
Loss: = 0.031
Loss: = 0.036
Loss: = 0.069
Loss: = 0.042
Loss: = 0.041
Loss: = 0.040
Loss: = 0.040
Loss: = 0.055
Loss: = 0.069
Loss: = 0.048
Loss: = 0.047
Loss: = 0.039
Loss: = 0.049
Loss: = 0.048
Loss: = 0.034
Loss: = 0.039
Loss: = 0.057
Loss: = 0.056
Loss: = 0.056
Loss: = 0.036
Loss: = 0.031
Loss: = 0.044
Loss: 

Loss: = 0.035
Loss: = 0.027
Loss: = 0.034
Loss: = 0.036
Loss: = 0.026
Loss: = 0.041
Loss: = 0.039
Loss: = 0.024
Loss: = 0.033
Loss: = 0.031
Loss: = 0.027
Loss: = 0.020
Loss: = 0.035
Loss: = 0.028
Loss: = 0.037
Loss: = 0.029
Loss: = 0.036
Loss: = 0.025
Loss: = 0.048

Epoch 21 ...
Loss: = 0.035
Loss: = 0.030
Loss: = 0.029
Loss: = 0.032
Loss: = 0.017
Loss: = 0.036
Loss: = 0.028
Loss: = 0.018
Loss: = 0.019
Loss: = 0.047
Loss: = 0.021
Loss: = 0.034
Loss: = 0.039
Loss: = 0.030
Loss: = 0.022
Loss: = 0.044
Loss: = 0.022
Loss: = 0.017
Loss: = 0.036
Loss: = 0.021
Loss: = 0.042
Loss: = 0.032
Loss: = 0.026
Loss: = 0.028
Loss: = 0.029
Loss: = 0.028
Loss: = 0.069
Loss: = 0.034
Loss: = 0.036
Loss: = 0.038
Loss: = 0.026
Loss: = 0.032
Loss: = 0.031
Loss: = 0.029
Loss: = 0.032
Loss: = 0.023
Loss: = 0.017
Loss: = 0.037
Loss: = 0.023
Loss: = 0.023
Loss: = 0.038
Loss: = 0.030
Loss: = 0.020
Loss: = 0.031
Loss: = 0.033
Loss: = 0.028
Loss: = 0.024
Loss: = 0.033
Loss: = 0.034
Loss: = 0.021
Loss: = 0.042
Loss: 